In [ ]:
import subprocess
from pathlib import Path

import geopandas
import pandas
from open_gira.geometry import buffer_via_reprojection
from open_gira.exposure import max_vector_rasters_intersection
from snail.damages import PiecewiseLinearDamageCurve

In [ ]:
base_dir = Path("../cluster")

In [ ]:
adm0 = geopandas.read_parquet(
    base_dir / "processed_data/admin-boundaries/admin-level-0.geoparquet"
).query('GID_0 == "THA"')

In [ ]:
tha_bounds = dict(adm0.bounds.iloc[0, :])
minx, miny, maxx, maxy = (
    tha_bounds["minx"],
    tha_bounds["miny"],
    tha_bounds["maxx"],
    tha_bounds["maxy"],
)
minx, miny, maxx, maxy

In [ ]:
raster_paths = sorted(Path("../results/input/jrc_flood/").glob("*.tif"))
for global_tiff in raster_paths:
    tha_tiff = base_dir / "processed_data" / "hazard" / global_tiff.name
    if tha_tiff.exists:
        continue
    print(tha_tiff)
    subprocess.check_output(
        [
            "gdalwarp",
            "-te",
            str(minx),
            str(miny),
            str(maxx),
            str(maxy),
            str(global_tiff),
            str(tha_tiff),
        ]
    )

In [ ]:
raster_paths = sorted(
    str(p) for p in (base_dir / "processed_data" / "hazard").glob("*.tif")
)
rasters = pandas.DataFrame(data={"path": raster_paths})
rasters["rp"] = rasters.path.str.extract("(\d+)").astype(int)
rasters["key"] = rasters.rp.apply(lambda rp: f"jrc_flood_{rp:03d}")
rasters.sort_values(by="rp", inplace=True)
rasters

In [ ]:
rail_edges = geopandas.read_parquet(
    base_dir
    / "transport_disruption"
    / "input"
    / "networks"
    / "rail"
    / "project-thailand"
    / "edges.gpq"
).query('from_iso_a3 == "THA" | to_iso_a3 == "THA"')
rail_edges.head(1)

In [ ]:
road_edges = geopandas.read_parquet(
    base_dir
    / "transport_disruption"
    / "input"
    / "networks"
    / "road"
    / "project-thailand"
    / "edges.gpq"
).query('from_iso_a3 == "THA" | to_iso_a3 == "THA"')
road_edges.head(1)

## Load powerplants

In [ ]:
powerplants = geopandas.read_parquet(
    base_dir / "power_flood" / "data" / "powerplants.geoparquet"
).cx[minx:maxx, miny:maxy]
powerplants["geom_point"] = powerplants.geometry
powerplants.geometry = buffer_via_reprojection(powerplants.geometry, 250)
powerplants.head(1)

In [ ]:
# US power plant construction costs
# https://www.eia.gov/electricity/generatorcosts/xls/generator_costs_2021.xlsx
# e.g. for gas...
# 920 [USD 2021] / capacity [kW]
# 920000 [USD 2021] / capacity [MW]
# N.B. not deflated to 2011, and no accounting for learning effects between 2011 and 2021

construction_cost_usd_per_mw = {
    "Biomass": 2592000,
    "Gas": 920000,
    "Solar": 1561000,
    "Wind": 1428000,
}
powerplants["construction_cost"] = powerplants.apply(lambda row: construction_cost_usd_per_mw[row.primary_fuel] * row.power_mw, axis=1)

## Powerplant RP damages

In [ ]:
powerplants = max_vector_rasters_intersection(powerplants, rasters).fillna(0)

In [ ]:
powerplants.head(2)

In [ ]:
damage_curves = {
    "paved": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_paved.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="central",
    ),
    "lower": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_paved.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="damage_fraction",
    ),
    "upper": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_paved.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="upper",
    ),
    "motorway": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_motorway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="damage_fraction",
    ),
    "motorway_high": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/road_motorway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="high flow",
    ),
    "rail_a": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/rail_railway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="damage_fraction",
    ),
    "rail_b": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/rail_railway.csv",
        intensity_col="inundation_depth_(m)",
        damage_col="high",
    ),
    "powerplants": PiecewiseLinearDamageCurve.from_csv(
        "../config/damage_curves/flood/powerplants.csv",
        intensity_col="depth_m",
        damage_col="damage_fraction"
    ),
}

In [ ]:
powerplants

In [ ]:
exposure = powerplants.loc[:, rasters.key]
damage_fraction = pandas.DataFrame(
    damage_curves["powerplants"].damage_fraction(exposure),
    index=exposure.index,
    columns=exposure.columns
)
damage_value = damage_fraction \
    .multiply(powerplants[fields.REHAB_COST], axis="index") \
    .multiply(powerplants[fields.SPLIT_LENGTH], axis="index")